In [1]:
#Packages to be installed
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# netcdf4 needs to be installed in your environment for this to work
import xarray as xr
import rioxarray 
import cartopy.crs as ccrs # For interactive mapping 
import cartopy.feature as cfeature
import seaborn as sns
import geopandas as gpd
import earthpy as et

In [11]:
# Importing dataset and visualising the datastructure
filepath_data1 = 'C:/SuSe2021/RemoteSensinginHydrology/Voluntary/GPM_IMERG/Processed/2001/*.nc4'
files_1 = glob.glob(filepath_data1)
dataset1 = xr.open_dataset(files_1[0])
dataset1

<xarray.Dataset>
Dimensions:                    (bnds: 2, lat: 8, lon: 10, time: 1)
Coordinates:
  * time                       (time) datetime64[ns] 2001-01-01
  * lon                        (lon) float32 84.62 84.88 85.12 ... 86.62 86.88
  * lat                        (lat) float32 26.62 26.88 27.12 ... 28.12 28.38
Dimensions without coordinates: bnds
Data variables:
    time_bnds                  (time, bnds) datetime64[ns] ...
    precipitationCal           (time, lon, lat) float32 ...
    precipitationCal_cnt       (time, lon, lat) int8 ...
    precipitationCal_cnt_cond  (time, lon, lat) int8 ...
    HQprecipitation            (time, lon, lat) float32 ...
    HQprecipitation_cnt        (time, lon, lat) int8 ...
    HQprecipitation_cnt_cond   (time, lon, lat) int8 ...
    randomError                (time, lon, lat) float32 ...
    randomError_cnt            (time, lon, lat) int8 ...
Attributes: (12/13)
    CDI:             Climate Data Interface version 1.9.3 (http://mpimet.mpg....
    Conventions:     CF-1.6
    history:         Thu Jul 22 17:26:53 2021: cdo -remapcon,C:/SuSe2021/Remo...
    BeginDate:       2001-01-01
    BeginTime:       00:00:00.000Z
    EndDate:         2001-01-01
    ...              ...
    FileHeader:      StartGranuleDateTime=2001-01-01T00:00:00.000Z;\nStopGran...
    InputPointer:    3B-HHR.MS.MRG.3IMERG.20010101-S000000-E002959.0000.V06B....
    title:           GPM IMERG Final Precipitation L3 1 day 0.1 degree x 0.1 ...
    DOI:             10.5067/GPM/IMERGDF/DAY/06
    ProductionTime:  2019-06-19T21:01:03.134Z
    CDO:             Climate Data Operators version 1.9.4rc1 (http://mpimet.m...

In [2]:
#Function to compare TRMM with CHIRPS
#filepath_data1 is the path to TRMM and filepath_data2 is the path to CHIRPS 
def evaluateTRMM(filepath_data1,filepath_data2):
    hit = 0
    false_alarm = 0
    miss = 0 
    null = 0
    files_1 = glob.glob(filepath_data1) #Listing files within folder
    files_2=glob.glob(filepath_data2)
    for j in range(0,len(files_2)):
        for i in range(0,len(files_1)):
            dataset1 = xr.open_dataset(files_1[i]) #Opening TRMM datasets
            ppt_dataset1 = np.array(dataset1['precipitation'][:]) #Accessing precipitation data
            dataset2 = xr.open_dataset(files_2[j]) #Opening CHIRPS datasets
            ppt_dataset2 = np.transpose(np.array(dataset2['precip'][i])) #Accessing precipitation data
            #Computing metrics 
            hit = hit + np.logical_and(ppt_dataset1 > 0, ppt_dataset2 >0)
            false_alarm = false_alarm + np.logical_and(ppt_dataset1 > 0, ppt_dataset2 == 0) 
            miss = miss + np.logical_and(ppt_dataset1 == 0, ppt_dataset2 > 0) 
            null = null + np.logical_and(ppt_dataset1 == 0, ppt_dataset2 == 0) 
    return np.sum(hit), np.sum(false_alarm), np.sum(miss), np.sum(null)

In [22]:
#Computing the no of hits, false, miss, null for different years
result_TRMM_2001 = evaluateTRMM('C:/SuSe2021/RemoteSensinginHydrology/Voluntary/TRMM_3B42/Processed/2001/*.nc4','C:/SuSe2021/RemoteSensinginHydrology/Voluntary/CHIRPS0_25/processed/2001/*.nc')
result_TRMM_2001

(7208, 4659, 1986, 15347)

In [4]:
result_TRMM_2002 = evaluateTRMM('C:/SuSe2021/RemoteSensinginHydrology/Voluntary/TRMM_3B42/Processed/2002/*.nc4','C:/SuSe2021/RemoteSensinginHydrology/Voluntary/CHIRPS0_25/processed/2002/*.nc')

In [5]:
result_TRMM_2002

(6809, 4889, 2155, 15347)

In [6]:
result_TRMM_2003 = evaluateTRMM('C:/SuSe2021/RemoteSensinginHydrology/Voluntary/TRMM_3B42/Processed/2003/*.nc4','C:/SuSe2021/RemoteSensinginHydrology/Voluntary/CHIRPS0_25/processed/2003/*.nc')

In [7]:
result_TRMM_2003

(7071, 4601, 1979, 15549)

In [14]:
#Function to evaluate GPM 
#filepath_data1 is the path to IMERG and filepath_data2 is the path to CHIRPS 
def evaluateGPM(filepath_data1,filepath_data2):
    hit = 0
    false_alarm = 0
    miss = 0 
    null = 0
    files_1 = glob.glob(filepath_data1)
    files_2=glob.glob(filepath_data2)
    for j in range(0,len(files_2)):
        for i in range(0,len(files_1)):
            dataset1 = xr.open_dataset(files_1[i])
            ppt_dataset1 = np.array(dataset1['precipitationCal'][:])
            dataset2 = xr.open_dataset(files_2[j])
            ppt_dataset2 = np.transpose(np.array(dataset2['precip'][i]))
            hit = hit + np.logical_and(ppt_dataset1 > 0, ppt_dataset2 >0)
            false_alarm = false_alarm + np.logical_and(ppt_dataset1 > 0, ppt_dataset2 == 0) 
            miss = miss + np.logical_and(ppt_dataset1 == 0, ppt_dataset2 > 0) 
            null = null + np.logical_and(ppt_dataset1 == 0, ppt_dataset2 == 0) 
    return np.sum(hit), np.sum(false_alarm), np.sum(miss), np.sum(null)

In [17]:
result_GPM_2001 = evaluateGPM('C:/SuSe2021/RemoteSensinginHydrology/Voluntary/GPM_IMERG/Processed/2001/*.nc4','C:/SuSe2021/RemoteSensinginHydrology/Voluntary/CHIRPS0_25/processed/2001/*.nc')

In [18]:
result_GPM_2001

(8771, 9265, 423, 10741)

In [19]:
result_GPM_2002 = evaluateGPM('C:/SuSe2021/RemoteSensinginHydrology/Voluntary/GPM_IMERG/Processed/2002/*.nc4','C:/SuSe2021/RemoteSensinginHydrology/Voluntary/CHIRPS0_25/processed/2002/*.nc')
result_GPM_2002

(8652, 10842, 312, 9394)

In [20]:
result_GPM_2003 = evaluateGPM('C:/SuSe2021/RemoteSensinginHydrology/Voluntary/GPM_IMERG/Processed/2003/*.nc4','C:/SuSe2021/RemoteSensinginHydrology/Voluntary/CHIRPS0_25/processed/2003/*.nc')
result_GPM_2003

(8809, 10827, 241, 9323)

In [25]:
TRMM_Hit =  result_TRMM_2001[0]+result_TRMM_2002[0]+result_TRMM_2003[0]

In [26]:
TRMM_False = result_TRMM_2001[1]+result_TRMM_2002[1]+result_TRMM_2003[1]

In [28]:
TRMM_Miss = result_TRMM_2001[2]+result_TRMM_2002[2]+result_TRMM_2003[2]

In [31]:
#Computation of metrics 
TRMM_POD = TRMM_Hit/(TRMM_Hit+TRMM_Miss)
TRMM_POD

0.7895839458982652

In [33]:
TRMM_FAR = TRMM_False/(TRMM_Hit+TRMM_False)
TRMM_FAR

0.48359414437152953

In [35]:
TRMM_CSI = TRMM_Hit/(TRMM_Hit+TRMM_Miss+TRMM_False)
TRMM_CSI

0.4539365253771711

In [36]:
GPM_Hit =  result_GPM_2001[0]+result_GPM_2002[0]+result_GPM_2003[0]
GPM_False = result_GPM_2001[1]+result_GPM_2002[1]+result_GPM_2003[1]
GPM_Miss = result_GPM_2001[2]+result_GPM_2002[2]+result_GPM_2003[2]

In [37]:
GPM_POD = GPM_Hit/(GPM_Hit+GPM_Miss)
GPM_POD

0.9641281975889444

In [38]:
GPM_FAR = GPM_False/(GPM_Hit+GPM_False)
GPM_FAR

0.5411258440331665

In [39]:
GPM_CSI = GPM_Hit/(GPM_Hit+GPM_Miss+GPM_False)
GPM_CSI

0.45117127033813764